# PM2.5台灣

<a id="load_data"></a>
## 上傳台灣2015一整年空氣監測資料 

### 步驟1: 將2015空氣監控資料，上傳至bluemix

In [1]:
def set_hadoop_config(credentials):
    prefix = "fs.swift.service." + credentials['name'] 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

### 步驟2: 將上傳資料與目前機器建立連結 `Insert to code` (畫面的右邊)

In [2]:
credentials_3 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_46719cd2_5b49_4783_8b86_f8a11f0314d4',
  'project_id':'646b1215221e4e7d8254de7bdf43bf1f',
  'region':'dallas',
  'user_id':'e0e7b9afb0884dfc8639dbf8606db94c',
  'domain_id':'a299c7413da54b60925ac070303b298e',
  'domain_name':'1159491',
  'username':'admin_ac4deff35b52ba280c7b60966e456e345cd3f69c',
  'password':"""jbE.e.rL^P9Z.9MC""",
  'filename':'pm2.5Taiwan.csv',
  'container':'notebooks',
  'tenantId':'sabe-89ac5efdbd68bd-a6554b527697'
}

### 步驟3: 將我們存放於open stack的資料集取名

In [3]:
credentials_3['name'] = 'pm25'
set_hadoop_config(credentials_3)

Now you can access the data by using the preconfigured `SparkContext` function in your notebook. 

First create a resilient distributed dataset ([`RDD`](https://spark.apache.org/docs/1.6.0/programming-guide.html#resilient-distributed-datasets-rdds)) of the raw data file. You can refer to it by using a URI of the form:

```
swift://container_name.name/object_name
```

If you didn't define a container name, at the time you created the Object Storage instance, the default container name that is used is `notebooks`. In this case, you can run the next code cell as it stands. Otherwise, change the container name. For `name` insert the Hadoop configuration name you defined, for example, `keystone`.

In [4]:
weather = sc.textFile("swift://notebooks.pm25/pm2.5Taiwan.csv")

### 步驟4: 試試看是否成功上傳  (使用count( ), first( ), collect( ), take( ) )

# 練習1: 讓我們求取2015年，大里每小時的平均pm25數值。
## 注意事項：
1. 資料分割：原始資料每一行為一個觀測值，我們必須將資料進行分割，才能逐一計算與進行操作。
2. 資料清洗：在氣象局的原始資料裡，有些數值由於當初偵測時有異常，所以會加註特別符號如\*\#等特殊符號，這些數值我們必須先經過前處理，我們才能進行算術運算。
3. 資料選擇：將大里資料挑選出來
4. 產生key-value，也就是(小時,pm25值)
5. 利用flatMap(), reduceByKey(), groupByKey()，將不同日期但相同時間的pm25值收集起來。
6. 計算平均值, 標準差, 最大最小值。


### 步驟一：資料分割 (使用map () 與 split( ))

### 步驟二：將大里站資料從全部資料集中挑選出來 (filter)

須留意unicode與string的差別,  u'大里'

### 步驟三：資料清洗 (使用 map(), str.strip())
#### 在氣象局的原始資料裡，有些數值由於當初偵測時有異常，所以會加註特別符號如\*\#等特殊符號，或者沒有取到數值為一空值，這些數值我們必須先經過前處理，我們才能進行算術運算。
1. ```2015/01/29 大里 PM2.5 14 14 12 7 1 0 4 6 12 16 15  52x 38x 30 29 27 29 24 24 21 19 19 23```
2. ```2015/01/16 大里 PM2.5 16 15 17 16 16 13 5 10 14 30 30 25 -4# 22 23 30 33 40 43 45 37 34 38 43``` 

### 步驟四：key value pair的產生 (重要的操作概念)
*將每小時資料轉成(小時,pm數值)，以求取每小時的平均值。

例如：
    2015/01/01 大里 PM2.5 53 55 58 53 43 36 35 42 55 64 65 59 52 44 47 41 43 40 42 35 28 20 18 16
    --> [(3, 53) (4, 55) (5, 58) (6, 53) (7, 43) (8, 36) (9, 35) (10, 42) (11, 55) (12, 64) (13, 65) (14, 59) (15, 52) (16, 44) (17, 47) (18, 41) (19, 43) (20, 40) (21, 42) (22, 35) (23, 28) (24, 20) (25, 18) (26, 16)]


### 步驟五： 利用flatMap(), reduceByKey(), groupByKey()，將不同日期但相同時間的pm25值收集起來。(使用flatMap)

### 步驟六： 計算大里區每個小時區間中，平均之pm25數值 (使用reduceByKey)

### 步驟七： 根據pm25平均濃度，進行排序。使用top( )

### 步驟八： 計算每個時間點的統計值，例如最大值、最小值、平均值、標準差(使用 groupByKey()與mapValues())

# 練習2: 請求取2015年，全國pm2.5最高的前十個工作站測點以及其日期。

# 練習3: 請算算看2015全國哪個測站，紫爆天數最多？
### 假設當日平均值大於60，則算該日該地區紫爆
